<a href="https://colab.research.google.com/github/sayan112207/Perplexa/blob/main/Perplexa_Research_Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arxiv==2.1.3 llama_index==0.12.3 llama-index-llms-mistralai==0.3.0 llama-index-embeddings-mistralai==0.3.0 gradio==3.39.0

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
from getpass import getpass
import requests
import sys
import arxiv
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, StorageContext, load_index_from_storage, PromptTemplate, Settings
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent

In [3]:
from google.colab import userdata
api_key = userdata.get('MISTRAL_API_KEY')

In [4]:
llm = MistralAI(api_key=api_key, model='mistral-large-latest')

In [5]:
model_name = "mistral-embed"
embed_model = MistralAIEmbedding(model_name=model_name, api_key=api_key)

In [6]:
def fetch_arxiv_papers(title :str, papers_count: int):
    search_query = f'all:"{title}"'
    search = arxiv.Search(
        query=search_query,
        max_results=papers_count,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = []
    # Use the Client for searching
    client = arxiv.Client()

    # Execute the search
    search = client.results(search)

    for result in search:
        paper_info = {
                'title': result.title,
                'authors': [author.name for author in result.authors],
                'summary': result.summary,
                'published': result.published,
                'journal_ref': result.journal_ref,
                'doi': result.doi,
                'primary_category': result.primary_category,
                'categories': result.categories,
                'pdf_url': result.pdf_url,
                'arxiv_url': result.entry_id
            }
        papers.append(paper_info)

    return papers

papers = fetch_arxiv_papers("Language Models", 10)

In [7]:
[[p['title']] for p in papers]

[['L$^2$M: Mutual Information Scaling Law for Long-Context Language Modeling'],
 ['LLMVoX: Autoregressive Streaming Text-to-Speech Model for Any LLM'],
 ['Shifting Long-Context LLMs Research from Input to Output'],
 ['Enough Coin Flips Can Make LLMs Act Bayesian'],
 ['Full-Duplex-Bench: A Benchmark to Evaluate Full-duplex Spoken Dialogue Models on Turn-taking Capabilities'],
 ['Predictable Scale: Part I -- Optimal Hyperparameter Scaling Law in Large Language Model Pretraining'],
 ['Scaling Rich Style-Prompted Text-to-Speech Datasets'],
 ['Universality of Layer-Level Entropy-Weighted Quantization Beyond Model Architecture and Size'],
 ['L1: Controlling How Long A Reasoning Model Thinks With Reinforcement Learning'],
 ['UIPE: Enhancing LLM Unlearning by Removing Knowledge Related to Forgetting Targets']]

In [8]:
def create_documents_from_papers(papers):
    documents = []
    for paper in papers:
        content = f"Title: {paper['title']}\n" \
                  f"Authors: {', '.join(paper['authors'])}\n" \
                  f"Summary: {paper['summary']}\n" \
                  f"Published: {paper['published']}\n" \
                  f"Journal Reference: {paper['journal_ref']}\n" \
                  f"DOI: {paper['doi']}\n" \
                  f"Primary Category: {paper['primary_category']}\n" \
                  f"Categories: {', '.join(paper['categories'])}\n" \
                  f"PDF URL: {paper['pdf_url']}\n" \
                  f"arXiv URL: {paper['arxiv_url']}\n"
        documents.append(Document(text=content))
    return documents



#Create documents for LlamaIndex
documents = create_documents_from_papers(papers)

In [9]:
Settings.chunk_size = 1024
Settings.chunk_overlap = 50

index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [10]:
index.storage_context.persist('index/')
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir='index/')

#load index
index = load_index_from_storage(storage_context, embed_model=embed_model)

In [11]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

rag_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="research_paper_query_engine_tool",
    description="A RAG engine with recent research papers.",
)

In [12]:
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template**Text:** 

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: response_synthesizer:refine_template**Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [13]:
def download_pdf(pdf_url, output_file):
    """
    Downloads a PDF file from the given URL and saves it to the specified file.

    Args:
        pdf_url (str): The URL of the PDF file to download.
        output_file (str): The path and name of the file to save the PDF to.

    Returns:
        str: A message indicating success or the nature of an error.
    """
    try:
        # Send a GET request to the PDF URL
        response = requests.get(pdf_url)
        response.raise_for_status()  # Raise an error for HTTP issues

        # Write the content of the PDF to the output file
        with open(output_file, "wb") as file:
            file.write(response.content)

        return f"PDF downloaded successfully and saved as '{output_file}'."

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

In [14]:
download_pdf_tool = FunctionTool.from_defaults(
    download_pdf,
    name='download_pdf_file_tool',
    description='python function, which downloads a pdf file by link'
)
fetch_arxiv_tool = FunctionTool.from_defaults(
    fetch_arxiv_papers,
    name='fetch_from_arxiv',
    description='download the {max_results} recent papers regarding the topic {title} from arxiv'
)

In [15]:
# building an ReAct Agent with the three tools.
agent = ReActAgent.from_tools([download_pdf_tool, rag_tool, fetch_arxiv_tool], llm=llm, verbose=True)

In [16]:
# create a prompt template to chat with an agent
q_template = (
    "I am interested in {topic}. \n"
    "Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to {topic}'. If there are not, could you fetch the recent one from arXiv? \n"
)

In [17]:
answer = agent.chat(q_template.format(topic="Audio-Language Models"))

> Running step be17b528-7d8c-4ee0-a2a7-5e7e4e494fb7. Step input: I am interested in Audio-Language Models. 
Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Audio-Language Models'. If there are not, could you fetch the recent one from arXiv? 

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, authors and link to download for papers related to Audio-Language Models'}
Observation: The title of the paper is "LLMVoX: Autoregressive Streaming Text-to-Speech Model for Any LLM." The authors are Sambal Shikhar, Mohammed Irfan Kurpath, Sahal Shaji Mullappilly, Jean Lahoud, Fahad Khan, Rao Muhammad Anwer, Salman Khan, and Hisham Cholakkal.

Summary:
This paper introduces LLMVoX, a lightwei

In [18]:
Markdown(answer.response)

I found a paper related to Audio-Language Models. Here are the details:

Title: LLMVoX: Autoregressive Streaming Text-to-Speech Model for Any LLM

Authors: Sambal Shikhar, Mohammed Irfan Kurpath, Sahal Shaji Mullappilly, Jean Lahoud, Fahad Khan, Rao Muhammad Anwer, Salman Khan, Hisham Cholakkal

Summary: This paper introduces LLMVoX, a lightweight, LLM-agnostic, autoregressive streaming Text-to-Speech (TTS) system designed to generate high-quality speech with low latency while preserving the base LLM's capabilities. LLMVoX addresses issues like fine-tuning requirements and text-speech misalignment, achieving a lower Word Error Rate and supporting seamless, infinite-length dialogues. Its plug-and-play design allows for easy extension to various tasks and languages, and it has been integrated with a Vision-Language Model to create an omni-model with speech, text, and vision capabilities.

You can download the paper here: http://arxiv.org/pdf/2503.04724v1

In [19]:
answer = agent.chat("Download the papers, which you mentioned above")

> Running step 1a6d442f-407b-416a-8069-dca138f09ead. Step input: Download the papers, which you mentioned above
Thought: I need to use a tool to help me answer the question.
Action: download_pdf_file_tool
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2503.04724v1', 'output_file': 'LLMVoX_Autoregressive_Streaming_Text_to_Speech_Model_for_Any_LLM.pdf'}
Observation: PDF downloaded successfully and saved as 'LLMVoX_Autoregressive_Streaming_Text_to_Speech_Model_for_Any_LLM.pdf'.
> Running step 3fc5a9ae-0c0c-4d41-9bd3-e2de42fe9d41. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The paper "LLMVoX: Autoregressive Streaming Text-to-Speech Model for Any LLM" has been downloaded successfully and saved as 'LLMVoX_Autoregressive_Streaming_Text_to_Speech_Model_for_Any_LLM.pdf'.


In [20]:
Markdown(answer.response)

The paper "LLMVoX: Autoregressive Streaming Text-to-Speech Model for Any LLM" has been downloaded successfully and saved as 'LLMVoX_Autoregressive_Streaming_Text_to_Speech_Model_for_Any_LLM.pdf'.

In [21]:
answer = agent.chat(q_template.format(topic="Min Max Similarity"))

> Running step d05aa99e-d676-4119-82d4-153d0091b374. Step input: I am interested in Min Max Similarity. 
Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Min Max Similarity'. If there are not, could you fetch the recent one from arXiv? 

Thought: I need to use a tool to help me answer the question.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, authors and link to download for papers related to Min Max Similarity'}
Observation: I'm sorry, but there are no papers related to "Min Max Similarity" in the given context information.
> Running step 863602f1-f120-494a-9665-c15b91099862. Step input: None
Thought: I need to use a tool to help me answer the question.
Action: fetch_from_arxiv
Action Input: {'title': 'Min Max Similarity', 'papers_count': 1}
Observation: [{'title': 'Channel Chart

In [22]:
Markdown(answer.response)

I found a recent paper related to Min Max Similarity from arXiv. Here are the details:

Title: Channel Charting for Streaming CSI Data

Authors: Sueda Taner, Maxime Guillaud, Olav Tirkkonen, Christoph Studer

Summary: This paper introduces a novel streaming channel charting (CC) architecture that maintains a small core CSI dataset for learning channel charts. The method uses a min-max-similarity criterion for curating the core CSI dataset, allowing it to approach the accuracy of the complete CSI dataset while using only a fraction of the storage and avoiding catastrophic forgetting of old CSI data. The self-supervised nature of CC enables predictive tasks that depend on user position without requiring any ground-truth position information.

You can download the paper here: http://arxiv.org/pdf/2312.04514v1

In [25]:
import gradio as gr

def research_agent(topic):
    """
    Function to handle user queries, interact with the agent,
    and return the agent's response.

    Args:
        topic (str): The user's research topic.

    Returns:
        str: The agent's response.
    """

    answer = agent.chat(q_template.format(topic=topic))  # Get the agent's response
    return (answer.response)  # Return the response

In [26]:
iface = gr.Interface(
    fn=research_agent,  # The function to handle queries
    inputs=gr.inputs.Textbox(lines=2, placeholder="Enter your research topic here..."),  # Input field
    outputs="text",  # Output format
    title="Research Paper Agent",  # Title of the interface
    description="Explore recent research papers on various topics.",  # Description
)

<ipython-input-26-308c233bc927>:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(lines=2, placeholder="Enter your research topic here..."),  # Input field
<ipython-input-26-308c233bc927>:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=2, placeholder="Enter your research topic here..."),  # Input field
<ipython-input-26-308c233bc927>:3: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=2, placeholder="Enter your research topic here..."),  # Input field


In [27]:
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://67848ace2fb4d16066.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


> Running step a702e91b-5b91-466a-afa2-9e8a56a3757f. Step input: I am interested in Large Language Models. 
Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Large Language Models'. If there are not, could you fetch the recent one from arXiv? 

Thought: (Implicit) I can answer without any more tools!
Answer: I found several papers related to Large Language Models. Here are the details:

1. **Title:** Shifting Long-Context LLMs Research from Input to Output
   **Authors:** Yuhao Wu, Yushi Bai, Zhiqing Hu, Shangqing Tu, Ming Shan Hee, Juanzi Li, Roy Ka-Wei Lee
   **Summary:** This paper advocates for a shift in NLP research towards addressing the challenges of long-output generation. Tasks like novel writing, long-term planning, and complex reasoning require models to produce coherent, contextually rich, and logically consisten